In [1]:
import pandas as pd
from pprint import pprint

# RecordReader

In [3]:
import xarray as xr
import json
from typing import Union
from pathlib import Path
import glob



def to_path(p: Union[str, Path]) -> Path:
    return p if isinstance(p, Path) else Path(p)

def find_records(path: str):
    search_path: str = f"{path}/**/signals/"
    all_paths = list(map(lambda x: str(to_path(x).parent), glob.glob(search_path, recursive=True)))
    return all_paths

class RecordReader():
    def __init__(self, path: Union[str, Path]):
        self.path = to_path(path)

    def load_signal(self, sig_name):
        return xr.open_zarr(self.path / "signals" / sig_name / "dataset")

    def load_signal_meta(self, sig_name):
        with open(self.path / "signals" / sig_name / "meta.json", "r") as meta:
            return json.load(meta)
    
    def load_metadata(self):
        with open(self.path / "meta.json", "r") as meta:
            return json.load(meta)

    def load_crf_metadata(self):
        with open(self.path / "crf.json", "r") as meta:
            return json.load(meta)

In [4]:
records = find_records(("./"))


In [5]:
len(records)

891

# Create a dataframes for signals and all data

In [6]:
data = {}
for r in records:
    reader = RecordReader(r)
    metadata = reader.load_metadata()
    scg_metadata = reader.load_signal_meta('scg-k')
    rsp_metadata = reader.load_signal_meta('rsp')
    crf_data = reader.load_crf_metadata()
    
    value = {
            'age': metadata['subject']['age']['value'],
            'sex' : metadata['subject']['sex'],
            'weight': metadata['subject']['weight']['value'],
            'height' : metadata['subject']['height']['value'],
            'subject_id' : crf_data['subject_id'],
            'study_id' : crf_data['study_id'],
            'hf_type' : crf_data['hf_type'],
            'sample_rate_scgk' : scg_metadata['sample_rate'],
            'nrg_lin_scgk' : reader.load_signal("scg-k").nrg.sel(motion="lin").to_pandas(),
            'nrg_rot_scgk' : reader.load_signal("scg-k").nrg.sel(motion="rot").to_pandas(),
            'pwr_lin_scgk': reader.load_signal("scg-k").pwr.sel(motion="lin").to_pandas(),
            'pwr_rot_scgk': reader.load_signal("scg-k").pwr.sel(motion="rot").to_pandas(),
            'sample_rate_rsp' : rsp_metadata['sample_rate'],
            'rsp': reader.load_signal("rsp").signal.to_pandas()
            }
    data[metadata['id']] = value

In [7]:
df = pd.DataFrame.from_dict(data, orient='index')

In [9]:
df.shape

(891, 14)

## Generating time domain and wavelet features

In [10]:
import numpy as np
from scipy.stats import skew, kurtosis
import pywt

def calculate_features(ts):
    mean = np.mean(ts)
    std = np.std(ts)
    median = np.median(ts)
    minimum = np.min(ts)
    maximum = np.max(ts)
    skewness = skew(ts)
    kurt = kurtosis(ts)
    rms = np.sqrt(np.mean(np.square(ts)))
    zero_crossings = np.sum(np.diff(np.sign(ts)) != 0)
    
    # Wavelet transformation
    wavelet = 'sym4'
    coeffs = pywt.wavedec(ts, wavelet, level=4)
    
    # Calculate wavelet features
    wavelet_mean = np.mean(np.concatenate(coeffs))
    wavelet_std = np.std(np.concatenate(coeffs))
    wavelet_energy = np.sum(np.square(np.concatenate(coeffs)))
    
    return [mean, std, median, minimum, maximum, skewness, kurt, rms, zero_crossings, wavelet_mean, wavelet_std, wavelet_energy]

# Compute time series features for each subject
for subject_id, value in data.items():
    for feature in ['nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk', 'pwr_rot_scgk', 'rsp']:
        ts = value[feature].values
        features = calculate_features(ts)
        
        # Store the computed features
        value[f"{feature}_mean"] = features[0]
        value[f"{feature}_std"] = features[1]
        value[f"{feature}_median"] = features[2]
        value[f"{feature}_min"] = features[3]
        value[f"{feature}_max"] = features[4]
        value[f"{feature}_skew"] = features[5]
        value[f"{feature}_kurt"] = features[6]
        value[f"{feature}_rms"] = features[7]
        value[f"{feature}_zero_crossings"] = features[8]
        value[f"{feature}_wavelet_mean"] = features[9]
        value[f"{feature}_wavelet_std"] = features[10]
        value[f"{feature}_wavelet_energy"] = features[11]

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index')



In [11]:
df.dtypes

age                   float64
sex                    object
weight                float64
height                float64
subject_id             object
                       ...   
rsp_rms               float64
rsp_zero_crossings      int32
rsp_wavelet_mean      float64
rsp_wavelet_std       float64
rsp_wavelet_energy    float64
Length: 74, dtype: object

In [12]:
# Checking target
df.hf_type.value_counts()

NoHF       477
UNKNOWN    282
HFrEF       85
HFpEF       27
HFmEF       20
Name: hf_type, dtype: int64

In [13]:
# Dropping the hf_type = UNKNOWN
df = df[df["hf_type"]!="UNKNOWN"]

In [14]:
# Check counts again
df.hf_type.value_counts()

NoHF     477
HFrEF     85
HFpEF     27
HFmEF     20
Name: hf_type, dtype: int64

In [15]:
from sklearn.preprocessing import LabelEncoder
# Encode the hf_type column as integer labels
encoder = LabelEncoder()
df['hf_type'] = encoder.fit_transform(df['hf_type'])

In [19]:
# Check counts again
df.hf_type.value_counts()

0    477
2     85
1     47
Name: hf_type, dtype: int64

In [17]:
# Creating a function to code HFpEF and HFmEF into one category, NoHF second category, and HFrEF third
def convert(df):
    if df["hf_type"]==3:
        return 0
    elif df["hf_type"]==2:
        return 2
    else:
        return 1

In [18]:
df["hf_type"] = df.apply(lambda df: convert(df), axis=1)

## Generating frequency features

In [20]:
from scipy.signal import periodogram
from scipy.stats import entropy

def spectral_entropy(pxx):
    psd_norm = pxx / np.sum(pxx)
    return entropy(psd_norm)

# Initialize a list to store the feature data
feature_data = []

# Iterate through the data dictionary
for key, value in data.items():
    # Calculate the periodogram for each time series
    freq_nrg_lin, pxx_nrg_lin = periodogram(value['nrg_lin_scgk'])
    freq_nrg_rot, pxx_nrg_rot = periodogram(value['nrg_rot_scgk'])
    freq_pwr_lin, pxx_pwr_lin = periodogram(value['pwr_lin_scgk'])
    freq_pwr_rot, pxx_pwr_rot = periodogram(value['pwr_rot_scgk'])
    freq_rsp, pxx_rsp = periodogram(value['rsp'])

    # Calculate the frequency-domain features for each time series
    features = {
        'record_id': key,
        'nrg_lin_mean_freq': np.mean(freq_nrg_lin),
        'nrg_lin_median_freq': np.median(freq_nrg_lin),
        'nrg_lin_peak_freq': freq_nrg_lin[np.argmax(pxx_nrg_lin)],
        'nrg_lin_spectral_entropy': spectral_entropy(pxx_nrg_lin),
        'nrg_rot_mean_freq': np.mean(freq_nrg_rot),
        'nrg_rot_median_freq': np.median(freq_nrg_rot),
        'nrg_rot_peak_freq': freq_nrg_rot[np.argmax(pxx_nrg_rot)],
        'nrg_rot_spectral_entropy': spectral_entropy(pxx_nrg_rot),
        'pwr_lin_mean_freq': np.mean(freq_pwr_lin),
        'pwr_lin_median_freq': np.median(freq_pwr_lin),
        'pwr_lin_peak_freq': freq_pwr_lin[np.argmax(pxx_pwr_lin)],
        'pwr_lin_spectral_entropy': spectral_entropy(pxx_pwr_lin),
        'pwr_rot_mean_freq': np.mean(freq_pwr_rot),
        'pwr_rot_median_freq': np.median(freq_pwr_rot),
        'pwr_rot_peak_freq': freq_pwr_rot[np.argmax(pxx_pwr_rot)],
        'pwr_rot_spectral_entropy': spectral_entropy(pxx_pwr_rot),
        'rsp_mean_freq': np.mean(freq_rsp),
        'rsp_median_freq': np.median(freq_rsp),
        'rsp_peak_freq': freq_rsp[np.argmax(pxx_rsp)],
        'rsp_spectral_entropy': spectral_entropy(pxx_rsp)
    }
    
    # Add the features to the feature_data list
    feature_data.append(features)

# Convert the feature_data list into a DataFrame
features_df = pd.DataFrame(feature_data)

In [21]:
features_df.head()

,record_id,nrg_lin_mean_freq,nrg_lin_median_freq,nrg_lin_peak_freq,nrg_lin_spectral_entropy,nrg_rot_mean_freq,nrg_rot_median_freq,nrg_rot_peak_freq,nrg_rot_spectral_entropy,pwr_lin_mean_freq,...,pwr_lin_peak_freq,pwr_lin_spectral_entropy,pwr_rot_mean_freq,pwr_rot_median_freq,pwr_rot_peak_freq,pwr_rot_spectral_entropy,rsp_mean_freq,rsp_median_freq,rsp_peak_freq,rsp_spectral_entropy
0,Frontiers_CP-01_CP-01_20210413-000000,0.25,0.25,0.002277,7.382609,0.25,0.25,0.002277,8.348998,0.25,...,0.025148,9.213835,0.25,0.25,0.040044,9.879472,0.25,0.25,0.000416,4.894895
1,Frontiers_CP-02_CP-02_20210413-000000,0.25,0.25,0.002592,7.718503,0.25,0.25,0.002592,9.763231,0.25,...,0.023480,9.614558,0.25,0.25,0.068769,10.072020,0.25,0.25,0.000532,4.359918
2,Frontiers_CP-03_CP-03_20210413-000000,0.25,0.25,0.000109,7.214435,0.25,0.25,0.000109,8.674756,0.25,...,0.023857,8.361892,0.25,0.25,0.033608,10.041811,0.25,0.25,0.000585,4.340069
3,Frontiers_CP-04_CP-04_20210413-000000,0.25,0.25,0.000004,8.455349,0.25,0.25,0.000004,9.045688,0.25,...,0.028346,10.193032,0.25,0.25,0.109253,10.290044,0.25,0.25,0.000886,4.932395
4,Frontiers_CP-05_CP-05_20210414-000000,0.25,0.25,0.000086,7.870620,0.25,0.25,0.000110,7.873324,0.25,...,0.022972,8.613639,0.25,0.25,0.071168,9.647862,0.25,0.25,0.000595,4.094955


In [22]:
# Set the index of features_df to be the record_id
features_df.set_index('record_id', inplace=True)

# Join the main DataFrame with the features_df
combined_df = pd.merge(df, features_df, left_index=True, right_index=True)


In [23]:
combined_df.head()

,age,sex,weight,height,subject_id,study_id,hf_type,sample_rate_scgk,nrg_lin_scgk,nrg_rot_scgk,...,pwr_lin_peak_freq,pwr_lin_spectral_entropy,pwr_rot_mean_freq,pwr_rot_median_freq,pwr_rot_peak_freq,pwr_rot_spectral_entropy,rsp_mean_freq,rsp_median_freq,rsp_peak_freq,rsp_spectral_entropy
Frontiers_CP-07_CP-07_20210414-000000,63.0,Female,55.0,1.58,CP-07,Frontiers,2,500,time 0 3.717964e-42 2 4.858295...,time 0 1.322109e-21 2 3.038149...,...,0.031442,9.177957,0.25,0.25,0.064219,9.810664,0.25,0.25,0.000535,4.034055
Frontiers_CP-25_CP-25_20210429-000000,73.0,Male,80.0,1.70,CP-25,Frontiers,2,500,time 0 4.395396e-42 2 1.656699...,time 0 3.101110e-21 2 4.808838...,...,0.022854,8.438479,0.25,0.25,0.022188,9.724399,0.25,0.25,0.000629,3.756682
Frontiers_CP-26_CP-26_20210430-000000,72.0,Male,52.0,1.69,CP-26,Frontiers,2,500,time 0 3.352892e-42 2 4.087231...,time 0 7.375647e-21 2 6.487727...,...,0.027830,9.132068,0.25,0.25,0.104303,10.346608,0.25,0.25,0.000681,4.096104
Frontiers_CP-32_CP-32_20210505-000000,66.0,Female,69.5,1.55,CP-32,Frontiers,2,500,time 0 2.669942e-41 2 2.060657...,time 0 1.168071e-20 2 1.303850...,...,0.019943,8.962660,0.25,0.25,0.112769,10.078464,0.25,0.25,0.000800,4.824584
Frontiers_CP-67_CP-67_20210716-000000,66.0,Female,73.0,1.50,CP-67,Frontiers,2,500,time 0 2.530986e-39 2 3.016467...,time 0 1.358777e-18 2 2.192057...,...,0.019805,9.155824,0.25,0.25,0.058725,9.725593,0.25,0.25,0.000758,4.138203


In [24]:
combined_df.columns

Index(['age', 'sex', 'weight', 'height', 'subject_id', 'study_id', 'hf_type',
       'sample_rate_scgk', 'nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk',
       'pwr_rot_scgk', 'sample_rate_rsp', 'rsp', 'nrg_lin_scgk_mean',
       'nrg_lin_scgk_std', 'nrg_lin_scgk_median', 'nrg_lin_scgk_min',
       'nrg_lin_scgk_max', 'nrg_lin_scgk_skew', 'nrg_lin_scgk_kurt',
       'nrg_lin_scgk_rms', 'nrg_lin_scgk_zero_crossings',
       'nrg_lin_scgk_wavelet_mean', 'nrg_lin_scgk_wavelet_std',
       'nrg_lin_scgk_wavelet_energy', 'nrg_rot_scgk_mean', 'nrg_rot_scgk_std',
       'nrg_rot_scgk_median', 'nrg_rot_scgk_min', 'nrg_rot_scgk_max',
       'nrg_rot_scgk_skew', 'nrg_rot_scgk_kurt', 'nrg_rot_scgk_rms',
       'nrg_rot_scgk_zero_crossings', 'nrg_rot_scgk_wavelet_mean',
       'nrg_rot_scgk_wavelet_std', 'nrg_rot_scgk_wavelet_energy',
       'pwr_lin_scgk_mean', 'pwr_lin_scgk_std', 'pwr_lin_scgk_median',
       'pwr_lin_scgk_min', 'pwr_lin_scgk_max', 'pwr_lin_scgk_skew',
       'pwr_lin_scgk_kurt'

## Modelling

First is just sample code (Random forest is used without any preproccesing just to see if code is functional)

In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# Encode the 'sex' column
#combined_df['sex'] = combined_df['sex'].map({'M': 0, 'F': 1})



# Drop the original columns containing DataFrames
# Split the dataset into features (X) and target (y)
X = combined_df.drop(['hf_type', 'study_id', 'subject_id', 'sex', 'nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk', 'pwr_rot_scgk', 'rsp'], axis=1)
y = combined_df['hf_type']

# Extract subject_id as groups
groups = combined_df['subject_id']

# Initialize the GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get the train and test indices based on the groups
train_idx, test_idx = next(gss.split(X, y, groups))

# Split the dataset into training and testing sets
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Initialize the classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# ROC auc
roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test), multi_class="ovr", average="macro")
print(f"Macro-averaged One-vs-Rest ROC AUC score:\n{roc_auc}")

Accuracy: 0.9029850746268657

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.98      0.95       112
           1       0.50      0.29      0.36         7
           2       0.90      0.60      0.72        15

    accuracy                           0.90       134
   macro avg       0.77      0.62      0.68       134
weighted avg       0.89      0.90      0.89       134

Macro-averaged One-vs-Rest ROC AUC score:
0.9564889428092879


XBG

In [28]:
import xgboost as xgb
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

# Split the dataset into features (X) and target (y)
X = combined_df.drop(['hf_type', 'study_id', 'subject_id', 'sex', 'nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk', 'pwr_rot_scgk', 'rsp'], axis=1)
y = combined_df['hf_type']

# Extract subject_id as groups
groups = combined_df['subject_id']

# Initialize the GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get the train and test indices based on the groups
train_idx, test_idx = next(gss.split(X, y, groups))

# Split the dataset into training and testing sets
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Determine the number of unique classes in the target variable
num_classes = y.nunique()

# Initialize the classifier
clf = xgb.XGBClassifier(
    objective='multi:softproba',
    num_class=num_classes,
    n_estimators=100,
    random_state=42,
    use_label_encoder=False,
)

# Train the classifier
clf.fit(X_train, y_train, eval_metric='mlogloss')

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# ROC auc
roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test), multi_class="ovr", average="macro")
print(f"Macro-averaged One-vs-Rest ROC AUC score:\n{roc_auc}")

F:\conda\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


Accuracy: 0.9328358208955224

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.98      0.97       112
           1       0.60      0.43      0.50         7
           2       0.86      0.80      0.83        15

    accuracy                           0.93       134
   macro avg       0.80      0.74      0.77       134
weighted avg       0.93      0.93      0.93       134

Macro-averaged One-vs-Rest ROC AUC score:
0.9496778519329468


## 1D CNN

In [30]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Split the dataset into features (X) and target (y)
X = combined_df.drop(['hf_type', 'study_id', 'subject_id', 'sex', 'nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk', 'pwr_rot_scgk', 'rsp'], axis=1)
y = combined_df['hf_type']

# Extract subject_id as groups
groups = combined_df['subject_id']

# Initialize the GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get the train and test indices based on the groups
train_idx, test_idx = next(gss.split(X, y, groups))

# Split the dataset into training and testing sets
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# One-hot encode the target variable
#encoder = LabelEncoder()
#y_train_enc = encoder.fit_transform(y_train)
#y_test_enc = encoder.transform(y_test)
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Reshape the input features to fit the 1D CNN model
max_length = X_train.shape[1]
X_train = X_train.reshape((X_train.shape[0], max_length, 1))
X_test = X_test.reshape((X_test.shape[0], max_length, 1))

# Define the model architecture
model = Sequential()

# Add the first convolutional block
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(max_length, 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# Add the second convolutional block
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(MaxPooling1D(pool_size=2))

# Add the third convolutional block
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(Dropout(rate=0.5))

# Flatten the output of the convolutional blocks
model.add(Flatten())

# Add the fully connected layers
model.add(Dense(units=1000, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1000, activation='relu'))
model.add(Dropout(rate=0.5))

# Add the output layer with Softmax activation function
model.add(Dense(units=3, activation='softmax'))

# Define the early stopping callback
early_stopping = EarlyStopping(patience=10, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True)

# Compile the model with categorical crossentropy loss
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the combined time series data
model.fit(X_train, y_train_cat, epochs=100, validation_data=(X_test, y_test_cat), callbacks=[early_stopping])


# Evaluate the model on the test set
y_pred_cat = model.predict(X_test)
y_pred = np.argmax(y_pred_cat, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Compute recall
recall = recall_score(y_test, y_pred, average=None)
print(f"Recall: {recall}")

# ROC auc
roc_auc = roc_auc_score(y_test, y_pred_cat, multi_class="ovr", average="macro")
print(f"Macro-averaged One-vs-Rest ROC AUC score:\n{roc_auc}")


Epoch 1/100
15/15 [==============================] - 3s 45ms/step - loss: 0.9977 - accuracy: 0.7053 - val_loss: 1.0683 - val_accuracy: 0.7687
Epoch 2/100
15/15 [==============================] - 0s 26ms/step - loss: 0.7571 - accuracy: 0.7389 - val_loss: 1.0079 - val_accuracy: 0.8433
Epoch 3/100
15/15 [==============================] - 0s 24ms/step - loss: 0.6941 - accuracy: 0.7368 - val_loss: 0.9799 - val_accuracy: 0.8433
Epoch 4/100
15/15 [==============================] - 0s 24ms/step - loss: 0.6549 - accuracy: 0.7621 - val_loss: 0.9578 - val_accuracy: 0.8433
Epoch 5/100
15/15 [==============================] - 0s 24ms/step - loss: 0.6050 - accuracy: 0.7389 - val_loss: 0.9135 - val_accuracy: 0.8433
Epoch 6/100
15/15 [==============================] - 0s 23ms/step - loss: 0.6230 - accuracy: 0.7684 - val_loss: 0.8655 - val_accuracy: 0.8433
Epoch 7/100
15/15 [==============================] - 0s 24ms/step - loss: 0.6170 - accuracy: 0.7747 - val_loss: 0.8664 - val_accuracy: 0.8433
Epoch 

## 5-fold CV

In [34]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold


num_classes = y_train.nunique()
num_folds = 5

# One-hot encode the target variables
y_cat = to_categorical(y)

# Define a function to create and return the model
def create_model():

    # Define the model architecture
    model = Sequential()

    # Add the first convolutional block
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(max_length, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    # Add the second convolutional block
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(MaxPooling1D(pool_size=2))

    # Add the third convolutional block
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
    model.add(Dropout(rate=0.5))

    # Flatten the output of the convolutional blocks
    model.add(Flatten())

    # Add the fully connected layers
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dropout(rate=0.5))

    # Add the output layer with Softmax activation function
    model.add(Dense(units=3, activation='softmax'))

    # Compile the model with categorical crossentropy loss
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
              
    return model

# Initialize the GroupKFold cross-validator
num_folds = 5
group_kfold = GroupKFold(n_splits=num_folds)

# Initialize variables to store evaluation metrics
accuracies = []
recalls = []
roc_aucs = []

# Iterate through the folds
for train_idx, val_idx in group_kfold.split(X, y, groups):
    # Split the dataset into training and validation sets for the current fold
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    # Standardize the input features
    scaler = StandardScaler()
    X_train_fold = scaler.fit_transform(X_train_fold)
    X_val_fold = scaler.transform(X_val_fold)

    # One-hot encode the target variable
    #y_train_fold_enc = encoder.transform(y_train_fold)
    #y_val_fold_enc = encoder.transform(y_val_fold)
    y_train_fold_cat = to_categorical(y_train_fold)
    y_val_fold_cat = to_categorical(y_val_fold)

    # Reshape the input features to fit the 1D CNN model
    X_train_fold = X_train_fold.reshape((X_train_fold.shape[0], max_length, 1))
    X_val_fold = X_val_fold.reshape((X_val_fold.shape[0], max_length, 1))

    # Create a new model for each fold
    model = create_model()

    # Train the model on the current fold
    model.fit(X_train_fold, y_train_fold_cat, epochs=100, validation_data=(X_val_fold, y_val_fold_cat), callbacks=[early_stopping], verbose=2)

    # Evaluate the model on the validation set
    y_pred_cat = model.predict(X_val_fold)
    y_pred = np.argmax(y_pred_cat, axis=1)
    accuracy = accuracy_score(y_val_fold, y_pred)
    accuracies.append(accuracy)

    recall = recall_score(y_val_fold, y_pred, average=None)
    recalls.append(recall)

    roc_auc = roc_auc_score(y_val_fold_cat, y_pred_cat, multi_class="ovr", average="macro")
    roc_aucs.append(roc_auc)

# Calculate and print mean and standard deviation of evaluation metrics
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
mean_recall = np.mean(recalls, axis=0)
std_recall = np.std(recalls, axis=0)
mean_roc_auc = np.mean(roc_aucs)
std_roc_auc = np.std(roc_aucs)

print(f"\nMean accuracy: {mean_accuracy:.4f}, Standard deviation: {std_accuracy:.4f}")
print(f"\nMean recall: {mean_recall}, Standard deviation: {std_recall}")
print(f"\nMean ROC AUC: {mean_roc_auc:.4f}, Standard deviation: {std_roc_auc:.4f}")

Epoch 1/100
16/16 - 3s - loss: 1.5073 - accuracy: 0.6838 - val_loss: 1.0085 - val_accuracy: 0.7869 - 3s/epoch - 181ms/step
Epoch 2/100
16/16 - 0s - loss: 0.7795 - accuracy: 0.7577 - val_loss: 0.9809 - val_accuracy: 0.8033 - 376ms/epoch - 24ms/step
Epoch 3/100
16/16 - 0s - loss: 0.6818 - accuracy: 0.7515 - val_loss: 0.9506 - val_accuracy: 0.8279 - 365ms/epoch - 23ms/step
Epoch 4/100
16/16 - 0s - loss: 0.6374 - accuracy: 0.7885 - val_loss: 0.9304 - val_accuracy: 0.8279 - 354ms/epoch - 22ms/step
Epoch 5/100
16/16 - 0s - loss: 0.5503 - accuracy: 0.7885 - val_loss: 0.9090 - val_accuracy: 0.8279 - 374ms/epoch - 23ms/step
Epoch 6/100
16/16 - 0s - loss: 0.6059 - accuracy: 0.7618 - val_loss: 0.7831 - val_accuracy: 0.8033 - 385ms/epoch - 24ms/step
Epoch 7/100
16/16 - 0s - loss: 0.5796 - accuracy: 0.7762 - val_loss: 0.8355 - val_accuracy: 0.8279 - 380ms/epoch - 24ms/step
Epoch 8/100
16/16 - 0s - loss: 0.5975 - accuracy: 0.7762 - val_loss: 0.8063 - val_accuracy: 0.8033 - 371ms/epoch - 23ms/step
Ep

## Converting the target variable to 0/1 (yes no HF)

In [49]:
def convert2(combined_df):
    if combined_df["hf_type"]==0:
        return 0
    
    else:
        return 1

In [50]:
combined_df["hf_type"] = combined_df.apply(lambda combined_df: convert2(combined_df), axis=1)

In [51]:
combined_df["hf_type"].value_counts()


0    477
1    132
Name: hf_type, dtype: int64

In [41]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Split the dataset into features (X) and target (y)
X = combined_df.drop(['hf_type', 'study_id', 'subject_id', 'sex', 'nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk', 'pwr_rot_scgk', 'rsp'], axis=1)
y = combined_df['hf_type']

# Extract subject_id as groups
groups = combined_df['subject_id']

# Initialize the GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get the train and test indices based on the groups
train_idx, test_idx = next(gss.split(X, y, groups))

# Split the dataset into training and testing sets
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# One-hot encode the target variable
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Reshape the input features to fit the 1D CNN model
max_length = X_train.shape[1]
X_train = X_train.reshape((X_train.shape[0], max_length, 1))
X_test = X_test.reshape((X_test.shape[0], max_length, 1))

# Define the model architecture
model = Sequential()

# Add the first convolutional block
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(max_length, 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))

# Add the second convolutional block
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(MaxPooling1D(pool_size=2))

# Add the third convolutional block
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(Dropout(rate=0.5))

# Flatten the output of the convolutional blocks
model.add(Flatten())

# Add the fully connected layers
model.add(Dense(units=1000, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1000, activation='relu'))
model.add(Dropout(rate=0.5))

# Add the output layer with Softmax activation function
model.add(Dense(units=2, activation='softmax'))

# Define the early stopping callback
early_stopping = EarlyStopping(patience=10, monitor='val_loss', mode='min', verbose=1, restore_best_weights=True)

# Compile the model with categorical crossentropy loss
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the combined time series data
model.fit(X_train, y_train_cat, epochs=100, validation_data=(X_test, y_test_cat), callbacks=[early_stopping])


# Evaluate the model on the test set
y_pred_cat = model.predict(X_test)
y_pred = np.argmax(y_pred_cat, axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Compute recall
recall = recall_score(y_test, y_pred, average=None)
print(f"Recall: {recall}")

# ROC auc
roc_auc = roc_auc_score(y_test, y_pred_cat[:,1])
print(f"Macro-averaged One-vs-Rest ROC AUC score:\n{roc_auc}")

Epoch 1/100
15/15 [==============================] - 3s 45ms/step - loss: 1.3651 - accuracy: 0.6632 - val_loss: 0.7109 - val_accuracy: 0.7761
Epoch 2/100
15/15 [==============================] - 0s 27ms/step - loss: 0.7164 - accuracy: 0.6968 - val_loss: 0.7048 - val_accuracy: 0.8358
Epoch 3/100
15/15 [==============================] - 0s 26ms/step - loss: 0.5702 - accuracy: 0.7389 - val_loss: 0.6519 - val_accuracy: 0.8433
Epoch 4/100
15/15 [==============================] - 0s 25ms/step - loss: 0.5568 - accuracy: 0.7453 - val_loss: 0.6488 - val_accuracy: 0.8433
Epoch 5/100
15/15 [==============================] - 0s 26ms/step - loss: 0.5309 - accuracy: 0.7474 - val_loss: 0.6046 - val_accuracy: 0.8433
Epoch 6/100
15/15 [==============================] - 0s 26ms/step - loss: 0.5210 - accuracy: 0.7453 - val_loss: 0.5740 - val_accuracy: 0.8507
Epoch 7/100
15/15 [==============================] - 0s 28ms/step - loss: 0.4833 - accuracy: 0.7726 - val_loss: 0.5514 - val_accuracy: 0.8507
Epoch 

In [47]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold


num_classes = y_train.nunique()
num_folds = 5

# Split the dataset into features (X) and target (y)
X = combined_df.drop(['hf_type', 'study_id', 'subject_id', 'sex', 'nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk', 'pwr_rot_scgk', 'rsp'], axis=1)
y = combined_df['hf_type']


# One-hot encode the target variables
y_cat = to_categorical(y)

# Define a function to create and return the model
def create_model():

    # Define the model architecture
    model = Sequential()

    # Add the first convolutional block
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(max_length, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))

    # Add the second convolutional block
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(MaxPooling1D(pool_size=2))

    # Add the third convolutional block
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
    model.add(Dropout(rate=0.5))

    # Flatten the output of the convolutional blocks
    model.add(Flatten())

    # Add the fully connected layers
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dropout(rate=0.5))

    # Add the output layer with Softmax activation function
    model.add(Dense(units=2, activation='softmax'))

    # Compile the model with categorical crossentropy loss
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
              
    return model

# Initialize the GroupKFold cross-validator
num_folds = 5
group_kfold = GroupKFold(n_splits=num_folds)

# Initialize variables to store evaluation metrics
accuracies = []
recalls = []
roc_aucs = []

# Iterate through the folds
for train_idx, val_idx in group_kfold.split(X, y, groups):
    # Split the dataset into training and validation sets for the current fold
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    # Standardize the input features
    scaler = StandardScaler()
    X_train_fold = scaler.fit_transform(X_train_fold)
    X_val_fold = scaler.transform(X_val_fold)

    # One-hot encode the target variable
    #y_train_fold_enc = encoder.transform(y_train_fold)
    #y_val_fold_enc = encoder.transform(y_val_fold)
    y_train_fold_cat = to_categorical(y_train_fold)
    y_val_fold_cat = to_categorical(y_val_fold)

    # Reshape the input features to fit the 1D CNN model
    X_train_fold = X_train_fold.reshape((X_train_fold.shape[0], max_length, 1))
    X_val_fold = X_val_fold.reshape((X_val_fold.shape[0], max_length, 1))

    # Create a new model for each fold
    model = create_model()

    # Train the model on the current fold
    model.fit(X_train_fold, y_train_fold_cat, epochs=100, validation_data=(X_val_fold, y_val_fold_cat), callbacks=[early_stopping], verbose=2)

    # Evaluate the model on the validation set
    y_pred_cat = model.predict(X_val_fold)
    y_pred = np.argmax(y_pred_cat, axis=1)
    accuracy = accuracy_score(y_val_fold, y_pred)
    accuracies.append(accuracy)

    recall = recall_score(y_val_fold, y_pred, average=None)
    recalls.append(recall)

    roc_auc = roc_auc_score(y_val_fold_cat, y_pred_cat)

    roc_aucs.append(roc_auc)

# Calculate and print mean and standard deviation of evaluation metrics
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
mean_recall = np.mean(recalls, axis=0)
std_recall = np.std(recalls, axis=0)
mean_roc_auc = np.mean(roc_aucs)
std_roc_auc = np.std(roc_aucs)

print(f"\nMean accuracy: {mean_accuracy:.4f}, Standard deviation: {std_accuracy:.4f}")
print(f"\nMean recall: {mean_recall}, Standard deviation: {std_recall}")
print(f"\nMean ROC AUC: {mean_roc_auc:.4f}, Standard deviation: {std_roc_auc:.4f}")

Epoch 1/100
16/16 - 2s - loss: 0.9471 - accuracy: 0.7166 - val_loss: 0.6571 - val_accuracy: 0.7787 - 2s/epoch - 155ms/step
Epoch 2/100
16/16 - 0s - loss: 0.6456 - accuracy: 0.7392 - val_loss: 0.6427 - val_accuracy: 0.8443 - 373ms/epoch - 23ms/step
Epoch 3/100
16/16 - 0s - loss: 0.5577 - accuracy: 0.7864 - val_loss: 0.6091 - val_accuracy: 0.8197 - 377ms/epoch - 24ms/step
Epoch 4/100
16/16 - 0s - loss: 0.5120 - accuracy: 0.7700 - val_loss: 0.6155 - val_accuracy: 0.8115 - 353ms/epoch - 22ms/step
Epoch 5/100
16/16 - 0s - loss: 0.5273 - accuracy: 0.7639 - val_loss: 0.5968 - val_accuracy: 0.7787 - 356ms/epoch - 22ms/step
Epoch 6/100
16/16 - 0s - loss: 0.4597 - accuracy: 0.7906 - val_loss: 0.5843 - val_accuracy: 0.7951 - 356ms/epoch - 22ms/step
Epoch 7/100
16/16 - 0s - loss: 0.4929 - accuracy: 0.7659 - val_loss: 0.5763 - val_accuracy: 0.7951 - 355ms/epoch - 22ms/step
Epoch 8/100
16/16 - 0s - loss: 0.4640 - accuracy: 0.7844 - val_loss: 0.5665 - val_accuracy: 0.7951 - 355ms/epoch - 22ms/step
Ep

In [ ]:
# Split the dataset into features (X) and target (y)
X = combined_df.drop(['hf_type', 'study_id', 'subject_id', 'sex', 'nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk', 'pwr_rot_scgk', 'rsp'], axis=1)
y = combined_df['hf_type']

# Extract subject_id as groups
groups = combined_df['subject_id']

# Initialize the GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get the train and test indices based on the groups
train_idx, test_idx = next(gss.split(X, y, groups))

# Split the dataset into training and testing sets
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# One-hot encode the target variable
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Reshape the input features to fit the 1D CNN model
max_length = X_train.shape[1]
X_train = X_train.reshape((X_train.shape[0], max_length, 1))
X_test = X_test.reshape((X_test.shape[0], max_length, 1))

In [58]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, LSTM, Dropout, BatchNormalization, Add, Activation, GlobalAveragePooling1D, Multiply, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def se_block(input, channels, se_ratio=16):
    squeeze = GlobalAveragePooling1D()(input)
    excitation = Dense(channels // se_ratio, activation='relu')(squeeze)
    excitation = Dense(channels, activation='sigmoid')(excitation)
    return Multiply()([input, excitation])

def residual_block(input, filters, pool=False):
    x = Conv1D(filters, 3, padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if pool:
        x = MaxPooling1D(pool_size=2, strides=2)(x)  # Add this line
    x = Conv1D(filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    if pool:
        input = MaxPooling1D(pool_size=2, strides=2)(input)  # Add this line
    x = Add()([x, input])
    x = Activation('relu')(x)
    return x

def type1_residual_stage(input, filters):
    x = residual_block(input, filters, pool=True)
    x = residual_block(x, filters)
    x = residual_block(x, filters)
    x = se_block(x, filters)
    dense_connection = Conv1D(64, 1)(input)
    dense_connection = MaxPooling1D(pool_size=2, strides=2)(dense_connection)
    x = tf.keras.layers.concatenate([x, dense_connection], axis=-1)
    return x

def type2_residual_stage(input, filters):
    x = residual_block(input, filters)
    x = residual_block(x, filters)
    x = residual_block(x, filters)
    x = se_block(x, filters)
    return x



def create_model(input_shape):
    input = Input(shape=input_shape)

    x = Conv1D(32, 1)(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = se_block(x, 32)

    x = type1_residual_stage(x, 32)
    x = type2_residual_stage(x, 32)
    x = type1_residual_stage(x, 32)
    x = type2_residual_stage(x, 32)
    x = type1_residual_stage(x, 32)
    x = type2_residual_stage(x, 128)

    x = LSTM(32, return_sequences=True, go_backwards=False)(x)
    x = LSTM(32, return_sequences=False, go_backwards=True)(x)

    fc_layers = [64, 32, 16, 8]
    for units in fc_layers:
        x = Dense(units, activation='relu')(x)
        x = Dropout(0.15)(x)

    output = Dense(3, activation='softmax')(x)

    model = Model(inputs=input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model




In [59]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical


# Split the dataset into features (X) and target (y)
X = combined_df.drop(['hf_type', 'study_id', 'subject_id', 'sex', 'nrg_lin_scgk', 'nrg_rot_scgk', 'pwr_lin_scgk', 'pwr_rot_scgk', 'rsp'], axis=1)
y = combined_df['hf_type']

# Extract subject_id as groups
groups = combined_df['subject_id']

# Initialize the GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Get the train and test indices based on the groups
train_idx, test_idx = next(gss.split(X, y, groups))

# Split the dataset into training and testing sets
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# One-hot encode the target variable
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_test_enc = encoder.transform(y_test)
y_train_cat = to_categorical(y_train_enc)
y_test_cat = to_categorical(y_test_enc)

# Reshape the input features to fit the 1D CNN model
max_length = X_train.shape[1]
X_train = X_train.reshape((X_train.shape[0], max_length, 1))
X_test = X_test.reshape((X_test.shape[0], max_length, 1))

# Create the model
input_shape = (max_length, 1)
model = create_model(input_shape)
model.summary()

# Train the model
num_epochs = 100  # Adjust this value based on your needs
batch_size = 32   # Adjust this value based on your needs
model.fit(X_train, y_train_cat, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test, y_test_cat))


ValueError: Inputs have incompatible shapes. Received shapes (42, 32) and (42, 96)